## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-01-17-36-30 +0000,nypost,Just end the suffering: Post reporter ready to...,https://nypost.com/2025/11/01/us-news/just-end...
1,2025-11-01-17-32-16 +0000,bbc,Two more charged over Louvre jewellery heist,https://www.bbc.com/news/articles/cvgkk1mkg0po...
2,2025-11-01-17-30-19 +0000,startribune,"Halloween overnight shooting kills one, injure...",https://www.startribune.com/halloween-night-sh...
3,2025-11-01-17-27-50 +0000,nypost,These new characters are gearing up for Macy’s...,https://nypost.com/2025/11/01/us-news/thes-new...
4,2025-11-01-17-19-25 +0000,nypost,"NJ inferno kills family of 5, including 3 kids...",https://nypost.com/2025/11/01/us-news/nj-blaze...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2390/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
55,trump,30
24,new,16
306,he,13
387,snap,12
540,shutdown,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
177,2025-10-31-23-04-00 +0000,nyt,Trump Administration Must Pay SNAP Benefits Du...,https://www.nytimes.com/2025/10/31/us/politics...,75
108,2025-11-01-08-43-08 +0000,nyt,Carney Says He Apologized to Trump Over Ad Rev...,https://www.nytimes.com/2025/11/01/world/canad...,73
245,2025-10-31-18-29-56 +0000,nypost,Canadian PM Mark Carney apologized for anti-ta...,https://nypost.com/2025/10/31/us-news/canadian...,72
37,2025-11-01-14-00-00 +0000,wsj,Canadian PM Carney Says He Apologized to Trump...,https://www.wsj.com/world/canadian-pm-carney-s...,71
215,2025-10-31-20-26-00 +0000,wsj,The federal government pays out $100 billion a...,https://www.wsj.com/politics/policy/what-to-kn...,71


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
177,75,2025-10-31-23-04-00 +0000,nyt,Trump Administration Must Pay SNAP Benefits Du...,https://www.nytimes.com/2025/10/31/us/politics...
215,60,2025-10-31-20-26-00 +0000,wsj,The federal government pays out $100 billion a...,https://www.wsj.com/politics/policy/what-to-kn...
49,54,2025-11-01-12-55-44 +0000,cbc,"'This is not a game,' Carney says as he signal...",https://www.cbc.ca/news/politics/carney-budget...
170,53,2025-10-31-23-24-00 +0000,wsj,The White House is imposing new restrictions o...,https://www.wsj.com/politics/policy/white-hous...
43,36,2025-11-01-13-14-15 +0000,nypost,"Andrew Cuomo, Curtis Sliwa closing gap on soci...",https://nypost.com/2025/11/01/us-news/andrew-c...
157,35,2025-11-01-00-09-02 +0000,nypost,JD Vance dresses up like meme-version of himse...,https://nypost.com/2025/10/31/us-news/jd-vance...
252,33,2025-10-31-18-12-27 +0000,bbc,'No inch here was spared': Hurricane Melissa d...,https://www.bbc.com/news/videos/ce3xxzg09gro?a...
245,33,2025-10-31-18-29-56 +0000,nypost,Canadian PM Mark Carney apologized for anti-ta...,https://nypost.com/2025/10/31/us-news/canadian...
213,28,2025-10-31-20-27-00 +0000,wsj,New York City Airports Disrupted by Air-Traffi...,https://www.wsj.com/business/airlines/new-york...
184,28,2025-10-31-22-15-54 +0000,nypost,Ukraine welcomes China’s offer to ‘help’ Trump...,https://nypost.com/2025/10/31/world-news/ukrai...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
